In [1]:
import tensorflow as tf
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants

In [2]:
export_dir = './saved'
graph_pb = 'model/har_lstm/frozen_har.pb'

In [9]:
builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_dir)

with tf.compat.v1.gfile.GFile(graph_pb, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

sigs = {}

with tf.compat.v1.Session(graph=tf.compat.v1.Graph()) as sess:
    # name="" is important to ensure we don't get spurious prefixing
    tf.compat.v1.import_graph_def(graph_def, name="")
    g = tf.compat.v1.get_default_graph()
    inp = g.get_tensor_by_name("Variable:0")
    out = g.get_tensor_by_name("^save/Assign_25:0")

    sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
        tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(
            {"in": inp}, {"out": out})

    builder.add_meta_graph_and_variables(sess,
                                         [tag_constants.SERVING],
                                         signature_def_map=sigs)

builder.save()

KeyError: "The name '^save/Assign_25:0' refers to a Tensor which does not exist. The operation, '^save/Assign_25', does not exist in the graph."